<a href="https://www.kaggle.com/code/paritkansal/data-creation-object-detection?scriptVersionId=219458283" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
pip install -q datasets pillow

# Cropped Image Creation

In [ ]:
from datasets import load_dataset

ds2 = load_dataset("naver-clova-ix/cord-v2")
ds = load_dataset("naver-clova-ix/cord-v1")

cord_v1 = ds.copy()
cord_v2 = ds2.copy()

cord_datasets = {"cord_v1": cord_v1, "cord_v2": cord_v2}

In [ ]:
from datasets import load_dataset
from PIL import Image
import matplotlib.pyplot as plt
import os
from tqdm.auto import tqdm

output_dir = "Cropped_images"
os.makedirs(output_dir, exist_ok=True)
padding = 23
def crop_and_save_images(dataset, output_dir, dataset_name):
    for split in tqdm(dataset.keys(), total = len(dataset.keys())):  # Iterate over 'train', 'validation', 'test' splits
        split_dir = os.path.join(output_dir, split)
        os.makedirs(split_dir, exist_ok=True)

        for idx, example in tqdm(enumerate(dataset[split]), total = len(dataset[split])):
            # print(example.keys())
            image = example['image']
            annotations = eval(example['ground_truth'])  # Contains bounding boxes
            width, height = image.size
            try:
                min_x1 = float('inf')
                min_y1 = float('inf')
                max_x3 = float('-inf')
                max_y3 = float('-inf')

                for line in annotations["valid_line"]:
                    for word in line["words"]:
                        quad = word["quad"]
                        min_x1 = min(min_x1, quad['x1'])
                        min_y1 = min(min_y1, quad['y1'])
                        max_x3 = max(max_x3, quad['x3'])
                        max_y3 = max(max_y3, quad['y3'])

                box = [min_x1, min_y1, max_x3, max_y3]

                padded_box = [
                    max(box[0] - (padding-5), 0),  # x1
                    max(box[1] - padding, 0),  # y1
                    min(box[2] + padding, width),  # x3
                    min(box[3] + padding, height)  # y3
                ]
                cropped_image = image.crop(padded_box)

                cropped_image_path = os.path.join(split_dir, f"{dataset_name}_{idx}.png")
                cropped_image.save(cropped_image_path)

            except Exception as e:
                print(f"Error processing image {image}: {e}")


for cord_dataset_name, cord_dataset in cord_datasets.items():
    crop_and_save_images(cord_dataset, output_dir, cord_dataset_name)
# crop_and_save_images(ds, output_dir, dataset_name)
print(f"All cropped images are saved in: {output_dir}")

# Background Image Creation

In [ ]:
import os
import shutil
import random

def move_random_images(folder_a, folder_b, n):
    # Ensure folder_b is writable
    if not os.path.exists(folder_b):
        os.makedirs(folder_b)
    
    # Check if the folder is in read-only file system, and handle accordingly
    if "/input" in folder_a:
        print("Detected read-only source folder, copying images to working directory...")
        writable_folder_a = "/kaggle/working/images"
        if not os.path.exists(writable_folder_a):
            os.makedirs(writable_folder_a)

        # Copy files to writable folder
        for file_name in os.listdir(folder_a):
            file_path = os.path.join(folder_a, file_name)
            if os.path.isfile(file_path):
                shutil.copy(file_path, writable_folder_a)
        folder_a = writable_folder_a
    
    # Get list of all images in folder A (writable directory now)
    all_images = [f for f in os.listdir(folder_a) if os.path.isfile(os.path.join(folder_a, f))]
    
    # Ensure we don't select more images than available
    if n > len(all_images):
        raise ValueError(f"Requested {n} images, but only {len(all_images)} available.")

    # Randomly select n unique images
    selected_images = random.sample(all_images, n)

    # Define dataset splits (4:1:1 ratio)
    num_train = int(n * 4 / 6)
    num_val = int(n * 1 / 6)
    num_test = n - (num_train + num_val)  # Ensures all images are used

    train_images = selected_images[:num_train]
    val_images = selected_images[num_train:num_train + num_val]
    test_images = selected_images[num_train + num_val:]

    # Ensure Folder B and its subdirectories exist
    for subfolder in ["train", "validation", "test"]:
        os.makedirs(os.path.join(folder_b, subfolder), exist_ok=True)

    # Move images to respective subfolders
    for img in train_images:
        shutil.move(os.path.join(folder_a, img), os.path.join(folder_b, "train", img))
    
    for img in val_images:
        shutil.move(os.path.join(folder_a, img), os.path.join(folder_b, "validation", img))
    
    for img in test_images:
        shutil.move(os.path.join(folder_a, img), os.path.join(folder_b, "test", img))

    print(f"Moved {n} images to {folder_b} (Train: {num_train}, Val: {num_val}, Test: {num_test})")

In [ ]:
import os
folder_a = "/kaggle/input/indoor-scenes-cvpr-2019"
if os.path.exists(folder_a):
    print(os.listdir(folder_a))  # Check if the folder contains any files
else:
    print(f"Folder not found: {folder_a}")

In [ ]:
folder_B = "/kaggle/working/Background"

folder_A = "/kaggle/input/indoor-scenes-cvpr-2019/indoorCVPR_09/Images/artstudio"
move_random_images(folder_A, folder_B, 100)

folder_A = "/kaggle/input/indoor-scenes-cvpr-2019/indoorCVPR_09/Images/auditorium"
move_random_images(folder_A, folder_B, 100)

folder_A = "/kaggle/input/indoor-scenes-cvpr-2019/indoorCVPR_09/Images/bathroom"
move_random_images(folder_A, folder_B, 200)

folder_A = "/kaggle/input/indoor-scenes-cvpr-2019/indoorCVPR_09/Images/bedroom"
move_random_images(folder_A, folder_B, 250)

folder_A = "/kaggle/input/indoor-scenes-cvpr-2019/indoorCVPR_09/Images/bookstore"
move_random_images(folder_A, folder_B, 200)

folder_A = "/kaggle/input/indoor-scenes-cvpr-2019/indoorCVPR_09/Images/children_room"
move_random_images(folder_A, folder_B, 250)

folder_A = "/kaggle/input/indoor-scenes-cvpr-2019/indoorCVPR_09/Images/classroom"
move_random_images(folder_A, folder_B, 300)

folder_A = "/kaggle/input/indoor-scenes-cvpr-2019/indoorCVPR_09/Images/corridor"
move_random_images(folder_A, folder_B, 300)

folder_A = "/kaggle/input/indoor-scenes-cvpr-2019/indoorCVPR_09/Images/dining_room"
move_random_images(folder_A, folder_B, 300)

folder_A = "/kaggle/input/indoor-scenes-cvpr-2019/indoorCVPR_09/Images/florist"
move_random_images(folder_A, folder_B, 200)

folder_A = "/kaggle/input/indoor-scenes-cvpr-2019/indoorCVPR_09/Images/kitchen"
move_random_images(folder_A, folder_B, 200)

folder_A = "/kaggle/input/indoor-scenes-cvpr-2019/indoorCVPR_09/Images/livingroom"
move_random_images(folder_A, folder_B, 500)

folder_A = "/kaggle/input/indoor-scenes-cvpr-2019/indoorCVPR_09/Images/meeting_room"
move_random_images(folder_A, folder_B, 250)

In [ ]:
import os

def count_files_in_folder(folder_b):
    total_files = 0
    
    for dirpath, dirnames, filenames in os.walk(folder_b):
        total_files += len(filenames)  # Count all files in the current directory

    return total_files

file_count = count_files_in_folder(folder_B)
print(f"Total number of files in {folder_B}: {file_count}")

In [ ]:
folder_path = '/kaggle/working/images'
shutil.rmtree(folder_path)

# Image Creation

In [ ]:
import os
import random
from PIL import Image, ImageDraw, ImageEnhance
import json


def generate_multiple_composites_with_custom_masks_and_distance(
        cropped_images_dir, output_dir, bg_images_dir,
        n_images, num_original_images, bg_color_images, bg_images_count,
        max_crop_images=9, canvas_size=(1000, 1000), min_distance=30, margin=20):
    """
    Generates multiple composite images with random placement of cropped images on a white canvas,
    ensuring minimum distance between images. Also generates corresponding masks with unique
    integer values for each image on a transparent background.
    
    Args:
        cropped_images_dir (str): Directory containing cropped images.
        output_dir (str): Directory to save the generated images and bounding boxes.
        bg_images_dir (str): Directory containing background images.
        n_images (int): Number of composite images to generate.
        num_original_images (int): Number of original images to add to the composite images.
        bg_color_images (int): Number of composite images with random background colors.
        bg_images_count (int): Number of composite images with background images.
        max_crop_images (int): Maximum number of cropped images to place in each composite image.
        canvas_size (tuple): Size of the canvas (width, height).
        min_distance (int): Minimum distance in pixels between placed images.
        margin (int): Margin (in pixels) to exclude from the mask.
    """
    count = 0
    
    # Create output directories
    ped_masks_dir = os.path.join(output_dir, "MasksImages")
    png_images_dir = os.path.join(output_dir, "PNGImages")
    os.makedirs(ped_masks_dir, exist_ok=True)
    os.makedirs(png_images_dir, exist_ok=True)
    
    # Load cropped images
    cropped_image_paths = [
        os.path.join(cropped_images_dir, f)
        for f in os.listdir(cropped_images_dir)
        if f.endswith((".png", ".jpg", ".jpeg"))
    ]
    
    # Load background images if provided
    bg_images = []
    if bg_images_dir:
        bg_images = [
            os.path.join(bg_images_dir, f)
            for f in os.listdir(bg_images_dir)
            if f.endswith((".png", ".jpg", ".jpeg"))
        ]
    
    # Filter images that fit within the canvas size
    valid_images = [img_path for img_path in cropped_image_paths if Image.open(img_path).size[0] <= canvas_size[0] and Image.open(img_path).size[1] <= canvas_size[1]]
    
    if not valid_images:
        raise ValueError("No valid images found that fit the canvas.")
    
    all_bounding_boxes = []
    
    # Generate composite images
    for i in range(n_images + bg_color_images + bg_images_count):
        if i % 100 == 0:
             print(i)
        
        # Create a blank canvas
        if i < n_images:
            canvas = Image.new("RGB", canvas_size, "white")
        elif i < n_images + bg_color_images:
            canvas = Image.new("RGB", canvas_size, tuple(random.randint(0, 255) for _ in range(3)))
        else:
            canvas = Image.open(random.choice(bg_images)).resize(canvas_size).convert("RGB")  # Ensure RGB mode
        
        mask_canvas = Image.new("L", canvas_size, 0)  # Mask canvas (grayscale, starts with 0)
        bounding_boxes = []
        mask_id = 1  # Start from 1, as 0 is the background
        
        # Select random cropped images
        selected_images = random.sample(valid_images, min(random.randint(1, max_crop_images), len(valid_images)))
        
        # Try placing images without overlap
        for image_path in selected_images:
            cropped_image = Image.open(image_path).convert("RGBA")
            
            # Apply enhancements (brightness, contrast)
            if i < (n_images + (bg_color_images // 2)):
                cropped_image = ImageEnhance.Brightness(cropped_image).enhance(1.4)
                cropped_image = ImageEnhance.Contrast(cropped_image).enhance(2.0)
                cropped_image = ImageEnhance.Brightness(cropped_image).enhance(1.2)
            
            # Apply random rotation (-5 to 5 degrees)
            rotated_image = cropped_image.rotate(random.uniform(-5, 5), expand=True)
            
            # Apply random scaling (between 0.7 and 1.3x)
            scale_factor1, scale_factor2 = random.uniform(0.7, 1.3), random.uniform(0.7, 1.3)
            new_width = min(canvas_size[0], int(rotated_image.size[0] * scale_factor1))
            new_height = min(canvas_size[1], int(rotated_image.size[1] * scale_factor2))
            resized_image = rotated_image.resize((new_width, new_height))
            
            # Try finding a valid non-overlapping position
            placed = False
            for _ in range(100):
                x_min, y_min = random.randint(0, canvas_size[0] - new_width), random.randint(0, canvas_size[1] - new_height)
                x_max, y_max = x_min + new_width, y_min + new_height
                
                if not any(x_min < box[2] + min_distance and x_max + min_distance > box[0] and y_min < box[3] + min_distance and y_max + min_distance > box[1] for box in bounding_boxes):
                    canvas.paste(resized_image, (x_min, y_min), resized_image)
                    for x in range(x_min, x_max):
                        for y in range(y_min, y_max):
                            if 0 <= x < canvas_size[0] and 0 <= y < canvas_size[1]:
                                mask_canvas.putpixel((x, y), mask_id)
                    bounding_boxes.append((x_min, y_min, x_max, y_max))
                    mask_id += 1
                    placed = True
                    break
        
        # Save composite and mask images
        composite_image_path = os.path.join(png_images_dir, f"generated_image_{count+1}.jpg")
        canvas = canvas.convert("RGB")  # Ensure compatible mode for JPEG
        canvas.save(composite_image_path, format="JPEG")
        mask_canvas.save(os.path.join(ped_masks_dir, f"masked_image_{count+1}.png"))
        count += 1
        
        all_bounding_boxes.append({"image": composite_image_path, "bounding_boxes": bounding_boxes})
    
    # Save original images
    for i in range(min(num_original_images, len(valid_images))):
        original_image_path = random.choice(valid_images)
        original_image = Image.open(original_image_path).convert("RGB")
        original_image_output_path = os.path.join(png_images_dir, f"generated_image_{count+1}.jpg")
        original_image.save(original_image_output_path)
        
        # Create mask for the original image
        mask = Image.new("L", original_image.size, 0)
        ImageDraw.Draw(mask).rectangle([margin, margin, original_image.size[0] - margin, original_image.size[1] - margin], fill=1)
        mask.save(os.path.join(ped_masks_dir, f"masked_image_{count+1}.png"))
        count += 1
    
    # Save bounding boxes to JSON
    with open(os.path.join(output_dir, "all_bounding_boxes.json"), 'w') as f:
        json.dump(all_bounding_boxes, f)

In [ ]:
cropped_images_dir = "/kaggle/working/Cropped_images/test"
output_dir = "/kaggle/working/Created_Images/test"
bg_images_dir = "/kaggle/working/Background/test"

generate_multiple_composites_with_custom_masks_and_distance(
        cropped_images_dir, output_dir, bg_images_dir,
        n_images = 900, num_original_images = 200, bg_color_images = 500, bg_images_count = 200,
        max_crop_images=9, canvas_size=(1000, 1000), min_distance=30, margin=20)

In [ ]:
cropped_images_dir = "/kaggle/working/Cropped_images/train"
output_dir = "/kaggle/working/Created_Images/train"
bg_images_dir = "/kaggle/working/Background/train"

generate_multiple_composites_with_custom_masks_and_distance(
        cropped_images_dir, output_dir, bg_images_dir,
        n_images = 900*4, num_original_images = 200*4, bg_color_images = 500*4, bg_images_count = 200*4,
        max_crop_images=9, canvas_size=(1000, 1000), min_distance=30, margin=20)

In [ ]:
cropped_images_dir = "/kaggle/working/Cropped_images/train"
output_dir = "/kaggle/working/Created_Images/validation"
bg_images_dir = "/kaggle/working/Background/validation"

generate_multiple_composites_with_custom_masks_and_distance(
        cropped_images_dir, output_dir, bg_images_dir,
        n_images = 900, num_original_images = 200, bg_color_images = 500, bg_images_count = 200,
        max_crop_images=9, canvas_size=(1000, 1000), min_distance=30, margin=20)

# Model Training

In [ ]:
!git clone https://github.com/pytorch/vision.git
%cd vision/references/detection

In [ ]:
import os
os.system("wget https://raw.githubusercontent.com/pytorch/vision/main/references/detection/engine.py")
os.system("wget https://raw.githubusercontent.com/pytorch/vision/main/references/detection/utils.py")
os.system("wget https://raw.githubusercontent.com/pytorch/vision/main/references/detection/coco_utils.py")
os.system("wget https://raw.githubusercontent.com/pytorch/vision/main/references/detection/coco_eval.py")
os.system("wget https://raw.githubusercontent.com/pytorch/vision/main/references/detection/transforms.py")

In [ ]:
import os
import torch

from torchvision.io import read_image
from torchvision.ops.boxes import masks_to_boxes
from torchvision import tv_tensors
from torchvision.transforms.v2 import functional as F


class PennFudanDataset(torch.utils.data.Dataset):
    def __init__(self, root, transforms):
        self.root = root
        self.transforms = transforms
        # load all image files, sorting them to
        # ensure that they are aligned
        self.imgs = list(sorted(os.listdir(os.path.join(root, "PNGImages"))))
        self.masks = list(sorted(os.listdir(os.path.join(root, "MasksImages"))))

    def __getitem__(self, idx):
        # load images and masks
        img_path = os.path.join(self.root, "PNGImages", self.imgs[idx])
        mask_path = os.path.join(self.root, "MasksImages", self.masks[idx])
        img = read_image(img_path)
        mask = read_image(mask_path)
        # instances are encoded as different colors
        obj_ids = torch.unique(mask)
        # first id is the background, so remove it
        obj_ids = obj_ids[1:]
        num_objs = len(obj_ids)

        # split the color-encoded mask into a set
        # of binary masks
        masks = (mask == obj_ids[:, None, None]).to(dtype=torch.uint8)

        # get bounding box coordinates for each mask
        boxes = masks_to_boxes(masks)

        # there is only one class
        labels = torch.ones((num_objs,), dtype=torch.int64)

        image_id = idx
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        # suppose all instances are not crowd
        iscrowd = torch.zeros((num_objs,), dtype=torch.int64)

        # Wrap sample and targets into torchvision tv_tensors:
        img = tv_tensors.Image(img)

        target = {}
        target["boxes"] = tv_tensors.BoundingBoxes(boxes, format="XYXY", canvas_size=F.get_size(img))
        target["masks"] = tv_tensors.Mask(masks)
        target["labels"] = labels
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = iscrowd

        if self.transforms is not None:
            img, target = self.transforms(img, target)

        return img, target

    def __len__(self):
        return len(self.imgs)

In [ ]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

# load a model pre-trained on COCO
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights="DEFAULT")

# replace the classifier with a new one, that has
# num_classes which is user-defined
num_classes = 2  # 1 class (person) + background
# get number of input features for the classifier
in_features = model.roi_heads.box_predictor.cls_score.in_features
# replace the pre-trained head with a new one
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

In [ ]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor


def get_model_instance_segmentation(num_classes):
    # load an instance segmentation model pre-trained on COCO
    model = torchvision.models.detection.maskrcnn_resnet50_fpn(weights="DEFAULT")

    # get number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    # now get the number of input features for the mask classifier
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    # and replace the mask predictor with a new one
    model.roi_heads.mask_predictor = MaskRCNNPredictor(
        in_features_mask,
        hidden_layer,
        num_classes
    )

    return model
import torch

In [ ]:
import torch
from torchvision.transforms import v2

def get_transform(train):
    transforms = []
    if train:
        # Using RandomShortestSize instead of RandomResize for compatibility
        transforms.append(v2.RandomShortestSize(min_size=800, max_size=1200))
        transforms.append(v2.RandomHorizontalFlip(0.5))  # Random horizontal flip with 50% probability
    # Updated as per warning: Use ToImage and ToDtype instead of deprecated ToTensor
    transforms.append(v2.ToImage())
    transforms.append(v2.ToDtype(torch.float32, scale=True))  # Scale=True normalizes the tensor
    return v2.Compose(transforms)

In [ ]:
import utils
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights="DEFAULT")
dataset_train = PennFudanDataset('/kaggle/working/Created_Images/test', get_transform(train=True))
data_loader = torch.utils.data.DataLoader(
    dataset_train,
    batch_size=2,
    shuffle=True,
    collate_fn=utils.collate_fn
)

# For Training
images, targets = next(iter(data_loader))
images = list(image for image in images)
targets = [{k: v for k, v in t.items()} for t in targets]
output = model(images, targets)  # Returns losses and detections
print(output)

# For inference
model.eval()
x = [torch.rand(3, 300, 400), torch.rand(3, 500, 400)]
predictions = model(x)  # Returns predictions
print(predictions[0])

In [ ]:
import torch
from engine import train_one_epoch, evaluate
import utils

# train on the GPU or on the CPU, if a GPU is not available
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# our dataset has two classes only - background and person
num_classes = 2

# use our dataset and defined transformations
dataset_train = PennFudanDataset('/kaggle/working/Created_Images/train', get_transform(train=True))
dataset_validation = PennFudanDataset('/kaggle/working/Created_Images/validation', get_transform(train=False))

indices = torch.randperm(len(dataset_train)).tolist()
dataset_train = torch.utils.data.Subset(dataset_train, indices)

indices = torch.randperm(len(dataset_validation)).tolist()
dataset_test = torch.utils.data.Subset(dataset_validation, indices)

# define training and validation data loaders
data_loader_train = torch.utils.data.DataLoader(
    dataset_train,
    batch_size=4,
    shuffle=True,
    collate_fn=utils.collate_fn
)
data_loader_test = torch.utils.data.DataLoader(
    dataset_test,
    batch_size=4,
    shuffle=False,
    collate_fn=utils.collate_fn
)

# get the model using our helper function
model = get_model_instance_segmentation(num_classes)

# move model to the right device
model.to(device)

# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(
    params,
    lr=0.005,
    momentum=0.9,
    weight_decay=0.0005
)

# and a learning rate scheduler
lr_scheduler = torch.optim.lr_scheduler.StepLR(
    optimizer,
    step_size=3,
    gamma=0.1
)

num_epochs = 6

for epoch in range(num_epochs):

    # Evaluate on the test dataset
    evaluate(model, data_loader_test, device=device)

    # Train for one epoch, printing every 10 iterations
    train_one_epoch(model, optimizer, data_loader_train, device, epoch, print_freq=100)

    # Update the learning rate
    lr_scheduler.step()

    # Save the model's state dict to Google Drive (recommended approach)
    torch.save(model.state_dict(), f"/kaggle/working/model_{epoch}.pth")

# Evaluate on the test dataset
evaluate(model, data_loader_test, device=device)

print("That's it!")

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

dataset_test = PennFudanDataset('/kaggle/working/Created_Images/test', get_transform(train=False))

indices = torch.randperm(len(dataset_test)).tolist()
dataset_test = torch.utils.data.Subset(dataset_test, indices)

data_loader_test = torch.utils.data.DataLoader(
    dataset_test,
    batch_size=4,
    shuffle=False,
    collate_fn=utils.collate_fn
)
evaluate(model, data_loader_test, device=device)